In [85]:
import pandas as pd
from scipy.stats import shapiro, f_oneway, kruskal, ttest_ind, mannwhitneyu
from statsmodels.stats.multitest import multipletests
from itertools import combinations

In [86]:
metric = "f1"
df = pd.read_csv(f"_out_table/{metric}_table_raw.csv", index_col=0)
df['Approach'] = df.index.astype(str) + ' ' + df['Approach']
df = df.reset_index(drop=True)
df
# merge die ersten beiden Spalten zu einer Spalte "Approach"# merge die ersten beiden Spalten zu einer Spalte "Approach"

,Approach,\# n_train_column_tasd,\# n_train_column_asqp,rest15_tasd_f1,rest15_asqp_f1,rest16_tasd_f1,rest16_asqp_f1,flightabsa_tasd_f1,flightabsa_asqp_f1,coursera_tasd_f1,coursera_asqp_f1,hotels_tasd_f1,hotels_asqp_f1,tasd_avg,asqp_avg
0,0 ['0-shot Gemma'],0,0,30.355097,24.733728,45.506257,28.964692,51.807229,42.371542,29.499072,13.356766,38.972810,23.018608,39.228093,26.489067
1,0 LLMA \textbackslash w Paraphrase,full,full,27.703719,18.537692,47.728798,26.072663,52.100361,42.577842,30.922514,13.405704,40.738341,26.265456,39.838747,25.371872
2,0 LLMA \textbackslash w DLO,full,full,25.817340,18.876697,47.407710,26.965652,51.754361,42.820006,30.601219,13.375756,40.704851,24.965851,39.257096,25.400792
3,10 ['k-shot Gemma'],10,10,54.467564,39.950372,66.746126,46.228123,60.356139,45.238095,41.693122,22.314911,56.514658,31.408776,55.955522,37.028056
4,10 Paraphrase,10,10,8.745620,1.321321,6.659462,3.560966,8.820961,3.439099,15.942500,4.745011,14.909699,2.634099,11.015649,3.140099
5,10 DLO,10,10,15.841343,4.365971,13.592447,5.178861,16.069869,4.871053,22.927928,4.473000,18.072061,3.531387,17.300730,4.484054
6,10 LLMA \textbackslash w Paraphrase,full,full,49.089373,35.035544,62.736040,44.592305,59.873443,45.892044,38.461295,22.470194,55.689604,32.827205,53.169951,36.163459
7,10 LLMA \textbackslash w DLO,full,full,49.233732,37.185741,62.371356,46.195826,61.397485,46.474790,39.221149,23.371349,55.271954,31.436163,53.499135,36.932774
8,10 EDA \textbackslash w DLO,110,110,30.181199,9.340943,19.159065,11.801499,22.663755,11.949079,28.828751,14.189199,25.654535,7.882942,25.297461,11.032732
9,10 QAIE,26.4,45.2,20.110322,9.963920,12.374344,14.780372,17.764307,15.382201,25.183107,17.543029,21.687704,9.856565,19.423957,13.505218


In [87]:
# create comparisons
approaches = df['Approach'].unique()
comparisons = list(combinations(approaches, 2))

# filter comparisons where comp[0] and comp[1] do not contain "QAIE"
comparisons = [comp for comp in comparisons if "QAIE" not in comp[0] and "QAIE" not in comp[1]]
comparisons = [comp for comp in comparisons if "DS2" not in comp[0] and "DS2" not in comp[1]]
comparisons = [comp for comp in comparisons if "EDA" not in comp[0] and "EDA" not in comp[1]]

comparisons = [comp for comp in comparisons if comp[0] != "10 DLO" and comp[1] != "10 DLO"]
comparisons = [comp for comp in comparisons if comp[0] != "50 DLO" and comp[1] != "50 DLO"]
comparisons = [comp for comp in comparisons if comp[0] != "10 Paraphrase" and comp[1] != "10 Paraphrase"]
comparisons = [comp for comp in comparisons if comp[0] != "50 Paraphrase" and comp[1] != "50 Paraphrase"]
comparisons = [comp for comp in comparisons if comp[0].startswith("10 ") and comp[1].startswith("10 ")] + [comp for comp in comparisons if comp[0].startswith("50 ") and comp[1].startswith("50 ")] + [comp for comp in comparisons if comp[0].startswith("0 ") and comp[1].startswith("0 ")] + [comp for comp in comparisons if comp[0].startswith("full ") or comp[1].startswith("full ")]
comparisons, len(comparisons)

([("10 ['k-shot Gemma']", '10 LLMA \\textbackslash w Paraphrase'),
  ("10 ['k-shot Gemma']", '10 LLMA \\textbackslash w DLO'),
  ('10 LLMA \\textbackslash w Paraphrase', '10 LLMA \\textbackslash w DLO'),
  ("50 ['k-shot Gemma']", '50 LLMA \\textbackslash w Paraphrase'),
  ("50 ['k-shot Gemma']", '50 LLMA \\textbackslash w DLO'),
  ('50 LLMA \\textbackslash w Paraphrase', '50 LLMA \\textbackslash w DLO'),
  ("0 ['0-shot Gemma']", '0 LLMA \\textbackslash w Paraphrase'),
  ("0 ['0-shot Gemma']", '0 LLMA \\textbackslash w DLO'),
  ('0 LLMA \\textbackslash w Paraphrase', '0 LLMA \\textbackslash w DLO'),
  ("0 ['0-shot Gemma']", 'full Paraphrase'),
  ("0 ['0-shot Gemma']", 'full DLO'),
  ('0 LLMA \\textbackslash w Paraphrase', 'full Paraphrase'),
  ('0 LLMA \\textbackslash w Paraphrase', 'full DLO'),
  ('0 LLMA \\textbackslash w DLO', 'full Paraphrase'),
  ('0 LLMA \\textbackslash w DLO', 'full DLO'),
  ("10 ['k-shot Gemma']", 'full Paraphrase'),
  ("10 ['k-shot Gemma']", 'full DLO'),
  ('10

In [88]:
# Perform two-sided t-tests for all comparisons with Bonferroni correction
tasd_p_values = []
asqp_p_values = []
comparison_labels = []

for a1, a2 in comparisons:

    tasd_values_a1 = df[df['Approach'] == a1][[f"{ds}_tasd_f1" for ds in ["rest15", "rest16", "flightabsa", "coursera", "hotels"]]].values.flatten().astype(float)
    tasd_values_a2 = df[df['Approach'] == a2][[f"{ds}_tasd_f1" for ds in ["rest15", "rest16", "flightabsa", "coursera", "hotels"]]].values.flatten().astype(float)
    asqp_values_a1 = df[df['Approach'] == a1][[f"{ds}_asqp_f1" for ds in ["rest15", "rest16", "flightabsa", "coursera", "hotels"]]].values.flatten().astype(float)
    asqp_values_a2 = df[df['Approach'] == a2][[f"{ds}_asqp_f1" for ds in ["rest15", "rest16", "flightabsa", "coursera", "hotels"]]].values.flatten().astype(float)
    
    # Two-sided t-test for TASD
    _, p_tasd = ttest_ind(tasd_values_a1, tasd_values_a2, alternative='two-sided')
    tasd_p_values.append(p_tasd)
    
    # Two-sided t-test for ASQP
    _, p_asqp = ttest_ind(asqp_values_a1, asqp_values_a2, alternative='two-sided')
    asqp_p_values.append(p_asqp)
    
    comparison_labels.append(f"{a1} vs {a2}")

# Combine all p-values for global Bonferroni correction
all_p_values = tasd_p_values + asqp_p_values
all_labels = [f"{label} TASD" for label in comparison_labels] + [f"{label} ASQP" for label in comparison_labels]

# Apply Bonferroni correction to all p-values
reject, all_corrected, _, _ = multipletests(all_p_values, method='holm')

# Split back into TASD and ASQP
tasd_corrected = all_corrected[:len(tasd_p_values)]
asqp_corrected = all_corrected[len(tasd_p_values):]

# Print results
print("TASD F1 Significance Tests (Two-sided t-test with global Bonferroni correction):")
for i, (label, p, corr) in enumerate(zip(comparison_labels, tasd_p_values, tasd_corrected)):
    a1, a2 = comparisons[i]
    tasd_values_a1 = df[df['Approach'] == a1][[f"{ds}_tasd_f1" for ds in ["rest15", "rest16", "flightabsa", "coursera", "hotels"]]].values.flatten().astype(float)
    tasd_values_a2 = df[df['Approach'] == a2][[f"{ds}_tasd_f1" for ds in ["rest15", "rest16", "flightabsa", "coursera", "hotels"]]].values.flatten().astype(float)
    mean_a1 = tasd_values_a1.mean()
    mean_a2 = tasd_values_a2.mean()
    adj_p = all_p_values[i] * len(all_p_values)  # Adjusted p-value
    sig_text = "signifikant" if adj_p < 0.05 else "nicht signifikant"
    print(f"{label}: Mittelwert {a1} = {mean_a1:.4f}, Mittelwert {a2} = {mean_a2:.4f}, p-value = {p:.4f}, p adj. = {adj_p:.4f}, {sig_text}")

print("\nASQP F1 Significance Tests (Two-sided t-test with global Bonferroni correction):")
for i, (label, p, corr) in enumerate(zip(comparison_labels, asqp_p_values, asqp_corrected)):
    a1, a2 = comparisons[i]
    asqp_values_a1 = df[df['Approach'] == a1][[f"{ds}_asqp_f1" for ds in ["rest15", "rest16", "flightabsa", "coursera", "hotels"]]].values.flatten().astype(float)
    asqp_values_a2 = df[df['Approach'] == a2][[f"{ds}_asqp_f1" for ds in ["rest15", "rest16", "flightabsa", "coursera", "hotels"]]].values.flatten().astype(float)
    mean_a1 = asqp_values_a1.mean()
    mean_a2 = asqp_values_a2.mean()
    adj_p = all_p_values[len(tasd_p_values) + i] * len(all_p_values)  # Adjusted p-value
    sig_text = "signifikant" if adj_p < 0.05 else "nicht signifikant"
    print(f"{label}: Mittelwert {a1} = {mean_a1:.4f}, Mittelwert {a2} = {mean_a2:.4f}, p-value = {p:.4f}, p adj. = {adj_p:.4f}, {sig_text}")

TASD F1 Significance Tests (Two-sided t-test with global Bonferroni correction):
10 ['k-shot Gemma'] vs 10 LLMA \textbackslash w Paraphrase: Mittelwert 10 ['k-shot Gemma'] = 55.9555, Mittelwert 10 LLMA \textbackslash w Paraphrase = 53.1700, p-value = 0.6543, p adj. = 36.6400, nicht signifikant
10 ['k-shot Gemma'] vs 10 LLMA \textbackslash w DLO: Mittelwert 10 ['k-shot Gemma'] = 55.9555, Mittelwert 10 LLMA \textbackslash w DLO = 53.4991, p-value = 0.6906, p adj. = 38.6717, nicht signifikant
10 LLMA \textbackslash w Paraphrase vs 10 LLMA \textbackslash w DLO: Mittelwert 10 LLMA \textbackslash w Paraphrase = 53.1700, Mittelwert 10 LLMA \textbackslash w DLO = 53.4991, p-value = 0.9582, p adj. = 53.6607, nicht signifikant
50 ['k-shot Gemma'] vs 50 LLMA \textbackslash w Paraphrase: Mittelwert 50 ['k-shot Gemma'] = 60.6045, Mittelwert 50 LLMA \textbackslash w Paraphrase = 57.1628, p-value = 0.5359, p adj. = 30.0098, nicht signifikant
50 ['k-shot Gemma'] vs 50 LLMA \textbackslash w DLO: Mittel